# estimation of repulsion strength after creation
Tim Tyree<br>
5.27.2024

In [1]:
import sys,os
og_stdout=sys.stdout
from lib.my_initialization import *
os.chdir(nb_dir)
from lib import *
# import cupy, cudf
# from lib.rapids_func import *
import seaborn as sns
import matplotlib.ticker as mtick
import warnings
warnings.simplefilter("ignore", UserWarning)
%load_ext autoreload
%autoreload 2

In [2]:
import seaborn as sns
#reset matplotlib
plt.plot([1]*4)
plt.close()
#reset matplotlib settings
import matplotlib as mpl
sns.reset_orig()
mpl.rc_file_defaults()

# define module

# estimate $b$ for the full models

\begin{equation}
b  = \frac{\langle MSR\rangle_{creat}}{\langle MSR\rangle_{annih}}(a+2D)  -2 D
\end{equation}

$$
\delta b^2 = \Big(\frac{a+2D}{S_a}\Big)^2\delta S_c^2
            +\Big(\frac{(a+2D)S_c}{S_a^2}\Big)^2\delta S_a^2
            +\Big(\frac{S_c}{S_a}\Big)^2\delta a^2
            +4\Big(\frac{S_c}{S_a}-1\Big)^2\delta D^2
$$

From the cardiac model simulations, we found $\langle MSR\rangle_{annih}=0.740\pm0.016$ cm$^2$ and $\langle MSR\rangle_{creat}=0.794\pm0.019$ cm$^2$ for the LR model and $\langle  MSR\rangle_{annih}=0.443\pm0.007$ cm$^2$ and $\langle MSR\rangle_{creat}=0.806\pm0.012$ cm$^2$ for the FK model.m

In [8]:
#LR
AMSRa = 0.740
Delta_AMSRa = 0.016  / 1.96 #std
AMSRc = 0.794
Delta_AMSRc = 0.019 / 1.96 #std
a = 9.3 
Delta_a = 0.3 / 1.96 #std
D = 0.42
Delta_D = 0.14 / 1.96 #std
Sa = AMSRa
Sc = AMSRc
b = (Sc/Sa)*(a+2*D) - 2*D
Delta_b = (((a+2*D)/Sa) ** 2) * (Delta_AMSRc ** 2)
Delta_b+= (((a+2*D)*Sc/(Sa**2)) ** 2) * (Delta_AMSRa ** 2)
Delta_b+= ((Sc/Sa) ** 2) * (Delta_a ** 2)
Delta_b+= 4 * ((Sc/Sa - 1) ** 2) * (Delta_D ** 2)
Delta_b = np.sqrt(Delta_b)
Delta_b*= 1.96
print(f"{b=:.4f} +/- {Delta_b:.4f} cm^2/s (LR)")
chi = b/a + 1
Delta_chi = np.sqrt ( (Delta_b/a) ** 2 + (b*Delta_a/(a**2)) ** 2 )
print(f"\t- {chi=:.4f} +/- {Delta_chi:.4f}")

#FK
AMSRa = 0.443
Delta_AMSRa = 0.007  / 1.96 #std
AMSRc = 0.806#
Delta_AMSRc = 0.012 / 1.96 #std
a = 1.552
Delta_a = 0.016 / 1.96 #std
D = 0.115
Delta_D = 0.008 / 1.96 #std
Sa = AMSRa
Sc = AMSRc
b = (Sc/Sa)*(a+2*D) - 2*D
Delta_b = (((a+2*D)/Sa) ** 2) * (Delta_AMSRc ** 2)
Delta_b+= (((a+2*D)*Sc/(Sa**2)) ** 2) * (Delta_AMSRa ** 2)
Delta_b+= ((Sc/Sa) ** 2) * (Delta_a ** 2)
Delta_b+= 4 * ((Sc/Sa - 1) ** 2) * (Delta_D ** 2)
Delta_b = np.sqrt(Delta_b)
Delta_b*= 1.96
print(f"{b=:.4f} +/- {Delta_b:.4f} cm^2/s (FK)")
chi = b/a + 1
Delta_chi = np.sqrt ( (Delta_b/a) ** 2 + (b*Delta_a/(a**2)) ** 2 )
print(f"\t- {chi=:.4f} +/- {Delta_chi:.4f}")

b=10.0399 +/- 0.4766 cm^2/s (LR)
	- chi=2.0796 +/- 0.0542
b=3.0122 +/- 0.0773 cm^2/s (FK)
	- chi=2.9408 +/- 0.0508


In [7]:
# b=10.0399 +/- 0.4766 cm^2/s (LR)
# b=3.0122 +/- 0.0773 cm^2/s (FK)

Consistency check:
$$b = a(\chi − 1)$$

$$\chi = b/a + 1$$

$$\delta \chi ^2 = \frac{\delta b^2}{a^2} + \frac{b^2 \delta a^2}{a^4}$$

# estimation for modified full models

In [14]:
printing=True

In [37]:
# #LR
# Ra_dir='/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_3.8_diffCoef_0.001/annihilation_ranges.csv'
# Rc_dir='/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_3.8_diffCoef_0.001/creation_ranges.csv'
#FK
Ra_dir='/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/param_set_8_ds_5.0_tmax_30_diffCoef_0.0005_dt_0.025_taud_0.54/annihilation_ranges.csv'
Rc_dir='/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/param_set_8_ds_5.0_tmax_30_diffCoef_0.0005_dt_0.025_taud_0.54/creation_ranges.csv'
df_ra = pd.read_csv(Ra_dir)
df_rc = pd.read_csv(Rc_dir)
df_ra['Rsq'] = df_ra['R'] ** 2
valuesa = df_ra.groupby(by=['trial_num','pid1'])['Rsq'].mean().values
df_rc['Rsq'] = df_rc['R'] ** 2
valuesc = df_rc.groupby(by=['trial_num','pid1'])['Rsq'].mean().values
# valuesa = (df_ra['R'] ** 2).values
# valuesc = (df_rc['R'] ** 2).values
# Sc = np.sum(df_rc['R'] ** 2)
# Sc/= df_rc.groupby(by=['trial_num','pid1']).ngroups
# Sc

# df_ra.head()

meana,Delta_meana,num_obsa,p_normala=comp_mean_bootstrap_uncertainty(valuesa)
meanc,Delta_meanc,num_obsc,p_normalc=comp_mean_bootstrap_uncertainty(valuesc)
if printing:
    print(f"FK:")
    print(f"mean annihilation value: {meana:.4f} +/- {Delta_meana:.4f} (N={num_obsa}, {p_normala=:.4f})")
    print(f"mean creation value:     {meanc:.4f} +/- {Delta_meanc:.4f} (N={num_obsc}, {p_normalc=:.4f})")
na = df_ra.groupby(by=['trial_num','pid1']).ngroups
nc = df_rc.groupby(by=['trial_num','pid1']).ngroups
print(f"num annihilation pairs: {na}")
print(f"num creation pairs: {nc}")

meana_fk,Delta_meana_fk,num_obsa_fk,p_normala_fk = meana,Delta_meana,num_obsa,p_normala
meanc_fk,Delta_meanc_fk,num_obsc_fk,p_normalc_fk = meanc,Delta_meanc,num_obsc,p_normalc

FK:
mean annihilation value: 0.5595 +/- 0.0080 (N=60673, p_normala=0.3408)
mean creation value:     0.8726 +/- 0.0110 (N=60364, p_normalc=0.2084)
num annihilation pairs: 60673
num creation pairs: 60364


From the cardiac model simulations, we found $\langle MSR\rangle_{annih}=0.740\pm0.016$ cm$^2$ and $\langle MSR\rangle_{creat}=0.794\pm0.019$ cm$^2$ for the LR model and $\langle  MSR\rangle_{annih}=0.443\pm0.007$ cm$^2$ and $\langle MSR\rangle_{creat}=0.806\pm0.012$ cm$^2$ for the FK model.

In [34]:
#LR
Ra_dir='/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_3.8_diffCoef_0.001/annihilation_ranges.csv'
Rc_dir='/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_3.8_diffCoef_0.001/creation_ranges.csv'
# #FK
# Ra_dir='/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/param_set_8_ds_5.0_tmax_30_diffCoef_0.0005_dt_0.025_taud_0.54/annihilation_ranges.csv'
# Rc_dir='/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/param_set_8_ds_5.0_tmax_30_diffCoef_0.0005_dt_0.025_taud_0.54/creation_ranges.csv'
df_ra = pd.read_csv(Ra_dir)
df_rc = pd.read_csv(Rc_dir)
df_ra['Rsq'] = df_ra['R'] ** 2
valuesa = df_ra.groupby(by=['trial_num','pid1'])['Rsq'].mean().values
df_rc['Rsq'] = df_rc['R'] ** 2
valuesc = df_rc.groupby(by=['trial_num','pid1'])['Rsq'].mean().values
# valuesa = (df_ra['R'] ** 2).values
# valuesc = (df_rc['R'] ** 2).values

meana,Delta_meana,num_obsa,p_normala=comp_mean_bootstrap_uncertainty(valuesa)
meanc,Delta_meanc,num_obsc,p_normalc=comp_mean_bootstrap_uncertainty(valuesc)
if printing:
    print("LR:")
    print(f"mean annihilation value: {meana:.4f} +/- {Delta_meana:.4f} (N={num_obsa}, {p_normala=:.4f})")
    print(f"mean creation value:     {meanc:.4f} +/- {Delta_meanc:.4f} (N={num_obsc}, {p_normalc=:.4f})")

na = df_ra.groupby(by=['trial_num','pid1']).ngroups
nc = df_rc.groupby(by=['trial_num','pid1']).ngroups
print(f"num annihilation pairs: {na}")
print(f"num creation pairs: {nc}")

meana_lr,Delta_meana_lr,num_obsa_lr,p_normala_lr = meana,Delta_meana,num_obsa,p_normala
meanc_lr,Delta_meanc_lr,num_obsc_lr,p_normalc_lr = meanc,Delta_meanc,num_obsc,p_normalc

LR:
mean annihilation value: 0.7527 +/- 0.0212 (N=10329, p_normala=0.8896)
mean creation value:     0.9027 +/- 0.0235 (N=10077, p_normalc=0.5579)
num annihilation pairs: 10329
num creation pairs: 10077


In [35]:
beep(3)



0

In [36]:
#LR
print(f"LR:")
AMSRa = meana_lr#??0.740
Delta_AMSRa = Delta_meana_lr#??0.016  / 1.96 #std
AMSRc = meanc_lr#??0.794
Delta_AMSRc = Delta_meanc_lr#??0.019 / 1.96 #std
a = 5.73#??9.3 
Delta_a = 0.07#??0.3 #/ 1.96 #std
D = 0.42#??0.42
Delta_D = 0.03#??0.14 #/ 1.96 #std
Sa = AMSRa
Sc = AMSRc
b = (Sc/Sa)*(a+2*D) - 2*D
Delta_b = (((a+2*D)/Sa) ** 2) * (Delta_AMSRc ** 2)
Delta_b+= (((a+2*D)*Sc/(Sa**2)) ** 2) * (Delta_AMSRa ** 2)
Delta_b+= ((Sc/Sa) ** 2) * (Delta_a ** 2)
Delta_b+= 4 * ((Sc/Sa - 1) ** 2) * (Delta_D ** 2)
Delta_b = np.sqrt(Delta_b)
# Delta_b*= 1.96
print(f"{b=:.4f} +/- {Delta_b:.4f} cm^2/s (LR)")
chi = b/a + 1
Delta_chi = np.sqrt ( (Delta_b/a) ** 2 + (b*Delta_a/(a**2)) ** 2 )
print(f"\t- {chi=:.4f} +/- {Delta_chi:.4f}")

#FK
print(f"FK:")
AMSRa = meana_fk#??0.443
Delta_AMSRa = Delta_meana_fk#??0.007  / 1.96 #std
AMSRc = meanc_fk#??0.806#
Delta_AMSRc = Delta_meanc_fk#??0.012 / 1.96 #std
a = 1.82#??1.552
Delta_a = 0.01#??0.016 #/ 1.96 #std
D = 0.048#??0.115
Delta_D = 0.007#??0.008 #/ 1.96 #std
Sa = AMSRa
Sc = AMSRc
b = (Sc/Sa)*(a+2*D) - 2*D
Delta_b = (((a+2*D)/Sa) ** 2) * (Delta_AMSRc ** 2)
Delta_b+= (((a+2*D)*Sc/(Sa**2)) ** 2) * (Delta_AMSRa ** 2)
Delta_b+= ((Sc/Sa) ** 2) * (Delta_a ** 2)
Delta_b+= 4 * ((Sc/Sa - 1) ** 2) * (Delta_D ** 2)
Delta_b = np.sqrt(Delta_b)
# Delta_b*= 1.96
print(f"{b=:.4f} +/- {Delta_b:.4f} cm^2/s (FK)")
chi = b/a + 1
Delta_chi = np.sqrt ( (Delta_b/a) ** 2 + (b*Delta_a/(a**2)) ** 2 )
print(f"\t- {chi=:.4f} +/- {Delta_chi:.4f}")

LR:
b=7.0387 +/- 0.3136 cm^2/s (LR)
	- chi=2.2284 +/- 0.0568
FK:
b=2.8923 +/- 0.0597 cm^2/s (FK)
	- chi=2.5892 +/- 0.0339
